In [12]:
import pysftp
import xarray as xr
import numpy as np
#import cartopy as cy
import matplotlib.pyplot as plt

from datetime import datetime
#from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER

import functions as fct
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Set figure size for all our plots
plt.rcParams['figure.figsize'] = [10., 8.]
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams.update({'font.size': 22})

In [14]:
# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
date = now.strftime("%Y%m%d")
time = now.strftime('%H')
print("date and time =", date, time)	

date and time = 20200220 13


In [15]:
ini_time = '09'

In [16]:
savefig = 0   # 1=yes, 0=no
form = 'png'

# File direction where figures should be saved (to be changed)
#figdir = 'sftp://franzihe@login.uio.no/uio/kant/geo-metos-u1/franzihe/www_docs'
figdir = '/home/franzihe/Documents/Figures/Weathermaps'

In [17]:
# Pick out area around 400km to Andenes
# Andøya Space Center Coordinates: 69.2950N, 16.0300E
# to be changed to the location of interest, define the lower left latitude and longitude,
# upper right corner latitude and longitude

#andenes_lat = 69.2950; andenes_lon = 16.03
#lower_lat = 65.69; lower_lon = 5.8
#upper_lat = 72.9;  upper_lon = 26.26
#lower_lat = 65.; lower_lon = 1.24
#upper_lat = 75;  upper_lon = 26.26

In [ ]:
# Nordmela Coordinates: 69.1358N, 15.6776E
andenes_lat = 69.135840; andenes_lon = 15.677645
lower_lat = 64.83; lower_lon = 0.98
upper_lat = 74.85;  upper_lon = 25.85

In [19]:
#  Open the netCDF file containing the input data.
# MEPS forecasts (Norway without Svalbard) from recent initialisations: https://thredds.met.no/thredds/catalog/mepslatest/catalog.html
# AROME Arctic forecast (Arctic including Svalbard) can be found in the 
# archive: https://thredds.met.no/thredds/catalog/aromearcticarchive/catalog.html
# latests: https://thredds.met.no/thredds/catalog/aromearcticlatest/catalog.html

#thredds = 'https://thredds.met.no/thredds/dodsC/mepslatest/meps_det_2_5km_%sT%sZ.ncml' %(date, ini_time)   # deterministic forecast
thredds = ''
fnx = xr.open_dataset(thredds, decode_times  = True, use_cftime = True)

In [20]:
lower_x, lower_y = fct.find_yx(fnx.latitude, fnx.longitude, lower_lat, lower_lon)
upper_x, upper_y = fct.find_yx(fnx.latitude, fnx.longitude, upper_lat, upper_lon)
andenes_x, andenes_y = fct.find_yx(fnx.latitude, fnx.longitude, andenes_lat, andenes_lon)

In [21]:
for forecast_in_hours in range(0,39,3):
    
    if forecast_in_hours < 10:
        forecast_in_hours = '0%s' %forecast_in_hours
#    print(int(forecast_in_hours))
    fig_name = 'meps_2_5_km_%sT%sZ_%s.%s' %(date,ini_time,forecast_in_hours, form)
#################################################################################
    map_area = 'Andoya'
    
#################################################################################
    ### 700hP - Temp - RH
    fct.plt_700_humidity(fnx, fnx.geopotential_pl.sel(pressure = 700.,
                                             x = slice(lower_x, upper_x),
                                             y = slice(lower_y, upper_y)).isel(time=int(forecast_in_hours))/100, 
                     fnx.air_temperature_pl.sel(pressure = 700.,
                                                x = slice(lower_x, upper_x),
                                                y = slice(lower_y, upper_y)).isel(time = int(forecast_in_hours)) - 273.15, 
                     fnx.relative_humidity_pl.sel(pressure = 700.,
                                                  x = slice(lower_x, upper_x),
                                                  y = slice(lower_y, upper_y)).isel(time = int(forecast_in_hours))*100, 
                     andenes_x, andenes_y)

    if savefig == 1:
        fct.createFolder('%s/700hPa_RH_T/%s/' %(figdir,map_area))
        plt.savefig('%s/700hPa_RH_T/%s/%s' %(figdir, map_area, fig_name), format = form, bbox_inches='tight', transparent=True)
        print('plot saved: %s/700hPa_RH_T/%s/%s' %(figdir, map_area, fig_name))
    plt.close()

#################################################################################    

    # convert Geopotential to height
    # https://en.wikipedia.org/wiki/Geopotential
    a = 6.378*10**6     # average radius of the earth  [m]
    G = 6.673*10**(-11) # gravitational constant       [Nm2/kg2]
    ma = 5.975*10**24   #  mass of the earth           [kg]

    Z_1000 = (-a**2 * fnx.geopotential_pl.sel(pressure = 1000.,).isel(time = int(forecast_in_hours)))/\
             (a * fnx.geopotential_pl.sel(pressure = 1000.,).isel(time = int(forecast_in_hours)) - G * ma)

    Z_500 = (-a**2 * fnx.geopotential_pl.sel(pressure = 500.,).isel(time = int(forecast_in_hours)))/\
             (a * fnx.geopotential_pl.sel(pressure = 500.,).isel(time = int(forecast_in_hours)) - G * ma)

    Z_thickness = (Z_500 - Z_1000)/10

    ########################################    
    ### Jet - Thickness - MSLP    
    fct.plt_Jet_Thick_MSLP(fnx, np.sqrt(fnx.x_wind_pl.sel(pressure = 250.,x = slice(lower_x, upper_x),y = slice(lower_y, upper_y)).isel(time = int(forecast_in_hours))**2 + \
                                        fnx.y_wind_pl.sel(pressure = 250.,x = slice(lower_x, upper_x),y = slice(lower_y, upper_y)).isel(time = int(forecast_in_hours))**2),
                       fnx.air_pressure_at_sea_level.sel(height_above_msl = 0, x = slice(lower_x, upper_x),y = slice(lower_y, upper_y)).isel(time = int(forecast_in_hours))/100, 
                       Z_thickness.sel(x = slice(lower_x, upper_x),y = slice(lower_y, upper_y)),
                       fnx.lwe_thickness_of_atmosphere_mass_content_of_water_vapor.sel(surface = 0, x = slice(lower_x, upper_x),y = slice(lower_y, upper_y)).isel(time = int(forecast_in_hours)),
                       andenes_x, andenes_y)
    if savefig == 1:
        fct.createFolder('%s/MSLP_Thickness_Jet/%s/' %(figdir,map_area))
        plt.savefig('%s/MSLP_Thickness_Jet/%s/%s' %(figdir, map_area, fig_name), format = form, bbox_inches='tight', transparent=True)
        print('plot saved: %s/MSLP_Thickness_Jet/%s/%s' %(figdir, map_area, fig_name))
    plt.close()
#################################################################################    

    ### 850hP - temperature - wind
    XX, YY = np.meshgrid(fnx.x.sel(x = slice(lower_x, upper_x)), 
                         fnx.y.sel(y = slice(lower_y, upper_y)))
    fct.plt_temp_wind_850(fnx, fnx.air_temperature_pl.sel(pressure = 850.,x = slice(lower_x, upper_x),y = slice(lower_y, upper_y)).isel(time = int(forecast_in_hours)) - 273.15,
                          fnx.x_wind_pl.sel(pressure = 850.,x = slice(lower_x, upper_x),y = slice(lower_y, upper_y)).isel(time = int(forecast_in_hours)),
                          fnx.y_wind_pl.sel(pressure = 850.,x = slice(lower_x, upper_x),y = slice(lower_y, upper_y)).isel(time = int(forecast_in_hours)),
                          XX, YY,
                          andenes_x, andenes_y)

    if savefig == 1:
        fct.createFolder('%s/850hPa_U_T/%s/' %(figdir,map_area))
        plt.savefig('%s/850hPa_U_T/%s/%s' %(figdir, map_area, fig_name), format = form, bbox_inches='tight', transparent=True)
        print('plot saved: %s/850hPa_U_T/%s/%s' %(figdir, map_area, fig_name))
    plt.close()
#################################################################################    


    map_area = 'Norway'
#################################################################################    

    ### 700hP - Temp - RH
    fct.plt_700_humidity(fnx, fnx.geopotential_pl.sel(pressure = 700.,).isel(time=int(forecast_in_hours))/100, 
                     fnx.air_temperature_pl.sel(pressure = 700.,).isel(time = int(forecast_in_hours)) - 273.15, 
                     fnx.relative_humidity_pl.sel(pressure = 700.,).isel(time = int(forecast_in_hours))*100, 
                     andenes_x, andenes_y)

    if savefig == 1:
        fct.createFolder('%s/700hPa_RH_T/%s/' %(figdir,map_area))
        plt.savefig('%s/700hPa_RH_T/%s/%s' %(figdir, map_area, fig_name), format = form, bbox_inches='tight', transparent=True)
        print('plot saved: %s/700hPa_RH_T/%s/%s' %(figdir, map_area, fig_name))
    plt.close()
#################################################################################    

    fct.plt_Jet_Thick_MSLP(fnx, np.sqrt(fnx.x_wind_pl.sel(pressure = 250.,).isel(time = int(forecast_in_hours))**2 + fnx.y_wind_pl.sel(pressure = 250.,).isel(time = int(forecast_in_hours))**2), 
                       fnx.air_pressure_at_sea_level.sel(height_above_msl = 0).isel(time = int(forecast_in_hours))/100, 
                       Z_thickness, 
                       fnx.lwe_thickness_of_atmosphere_mass_content_of_water_vapor.sel(surface = 0).isel(time = int(forecast_in_hours)), 
                       andenes_x, andenes_y)

    if savefig == 1:
        fct.createFolder('%s/MSLP_Thickness_Jet/%s/' %(figdir,map_area))
        plt.savefig('%s/MSLP_Thickness_Jet/%s/%s' %(figdir, map_area, fig_name), format = form, bbox_inches='tight', transparent=True)
        print('plot saved: %s/MSLP_Thickness_Jet/%s/%s' %(figdir, map_area, fig_name))
    plt.close()
#################################################################################

    XX, YY = np.meshgrid(fnx.x, fnx.y)
    fct.plt_temp_wind_850(fnx, fnx.air_temperature_pl.sel(pressure = 850,).isel(time = int(forecast_in_hours)) - 273.15,
                          fnx.x_wind_pl.sel(pressure = 850,).isel(time = int(forecast_in_hours)),
                          fnx.y_wind_pl.sel(pressure = 850,).isel(time = int(forecast_in_hours)),
                          XX, YY,
                          andenes_x, andenes_y)

    if savefig == 1:
        fct.createFolder('%s/850hPa_U_T/%s/' %(figdir,map_area))
        plt.savefig('%s/850hPa_U_T/%s/%s' %(figdir, map_area, fig_name), format = form, bbox_inches='tight', transparent=True)
        print('plot saved: %s/850hPa_U_T/%s/%s' %(figdir, map_area, fig_name))
    plt.close()

plot saved: /home/franzihe/Documents/Figures/Weathermaps/700hPa_RH_T/Andoya/meps_2_5_km_20200220T09Z_00.png


/home/franzihe/anaconda3/envs/weathermap/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


plot saved: /home/franzihe/Documents/Figures/Weathermaps/MSLP_Thickness_Jet/Andoya/meps_2_5_km_20200220T09Z_00.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/850hPa_U_T/Andoya/meps_2_5_km_20200220T09Z_00.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/700hPa_RH_T/Norway/meps_2_5_km_20200220T09Z_00.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/MSLP_Thickness_Jet/Norway/meps_2_5_km_20200220T09Z_00.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/850hPa_U_T/Norway/meps_2_5_km_20200220T09Z_00.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/700hPa_RH_T/Andoya/meps_2_5_km_20200220T09Z_03.png


/home/franzihe/anaconda3/envs/weathermap/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


plot saved: /home/franzihe/Documents/Figures/Weathermaps/MSLP_Thickness_Jet/Andoya/meps_2_5_km_20200220T09Z_03.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/850hPa_U_T/Andoya/meps_2_5_km_20200220T09Z_03.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/700hPa_RH_T/Norway/meps_2_5_km_20200220T09Z_03.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/MSLP_Thickness_Jet/Norway/meps_2_5_km_20200220T09Z_03.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/850hPa_U_T/Norway/meps_2_5_km_20200220T09Z_03.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/700hPa_RH_T/Andoya/meps_2_5_km_20200220T09Z_06.png


...

plot saved: /home/franzihe/Documents/Figures/Weathermaps/MSLP_Thickness_Jet/Andoya/meps_2_5_km_20200220T09Z_30.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/850hPa_U_T/Andoya/meps_2_5_km_20200220T09Z_30.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/700hPa_RH_T/Norway/meps_2_5_km_20200220T09Z_30.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/MSLP_Thickness_Jet/Norway/meps_2_5_km_20200220T09Z_30.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/850hPa_U_T/Norway/meps_2_5_km_20200220T09Z_30.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/700hPa_RH_T/Andoya/meps_2_5_km_20200220T09Z_33.png


/home/franzihe/anaconda3/envs/weathermap/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


plot saved: /home/franzihe/Documents/Figures/Weathermaps/MSLP_Thickness_Jet/Andoya/meps_2_5_km_20200220T09Z_33.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/850hPa_U_T/Andoya/meps_2_5_km_20200220T09Z_33.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/700hPa_RH_T/Norway/meps_2_5_km_20200220T09Z_33.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/MSLP_Thickness_Jet/Norway/meps_2_5_km_20200220T09Z_33.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/850hPa_U_T/Norway/meps_2_5_km_20200220T09Z_33.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/700hPa_RH_T/Andoya/meps_2_5_km_20200220T09Z_36.png


/home/franzihe/anaconda3/envs/weathermap/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


plot saved: /home/franzihe/Documents/Figures/Weathermaps/MSLP_Thickness_Jet/Andoya/meps_2_5_km_20200220T09Z_36.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/850hPa_U_T/Andoya/meps_2_5_km_20200220T09Z_36.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/700hPa_RH_T/Norway/meps_2_5_km_20200220T09Z_36.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/MSLP_Thickness_Jet/Norway/meps_2_5_km_20200220T09Z_36.png
plot saved: /home/franzihe/Documents/Figures/Weathermaps/850hPa_U_T/Norway/meps_2_5_km_20200220T09Z_36.png


In [ ]:
fnx.close()